In [ ]:
# Import dependencies
import json
import pandas as pd
import requests
import time

In [ ]:
# Define parameters for query
begin_date = '2013-01-01T00:00:00.000' # Enter the YYYY-MM-DD, leave T00:00:00.000 intact
end_date = '2018-12-31T00:00:00.000'
borough = 'MANHATTAN' #Choose one of the five boroughs

# This query URL will get all rows based on date range
base_query = f"https://data.cityofnewyork.us/resource/qgea-i56i.json?$where=rpt_dt%20between%20%27{begin_date}%27%20and%20%27{end_date}%27"

# This query will search the same date range but also filter by borough -- this is the one we're going to use
borough_query = f"https://data.cityofnewyork.us/resource/qgea-i56i.json?$where=boro_nm%20=%20%27{borough}%27%20and%20rpt_dt%20between%20%27{begin_date}%27%20and%20%27{end_date}%27"

In [ ]:
# Create lists to store fields from the JSON
complaint_number = [] #'cmplnt_num'
reported_date = [] #'rpt_dt'
borough_name = [] #'boro_nm'
offense_desc = [] #'ofns_desc'
pd_desc = [] # 'pd_desc'
law_cat = [] #law_cat_cd
lat = [] #latitude
lon = [] #longitude

In [ ]:
# Define variables for the For Loop.
records_count = 1
records = 0
offset = 0
set = 1
limit = 1000

In [ ]:
# While loop. While len of json_results > 0, query the URL.
while records_count != 0:
    # Define the limit/offset portion of the URL. Updates after every iteration. The limit is default set to 1000, per API documentation.
    limit_query = f"&$limit=1000&$offset={offset}"
    # Get the JSON
    json_results = requests.get(borough_query,limit_query).json()
    # Add to the number of results to track total number
    records += len(json_results)
    # Print message to console showing number of rows extracted, show set number.
    print(f"Extracting {len(json_results)} rows of data, Set # {set}")
    # Extract and store requested data into lists
    for x in range(len(json_results)):
        complaint_number.append(json_results[x]['cmplnt_num'])
        reported_date.append(json_results[x]['rpt_dt'])
        borough_name.append(json_results[x]['boro_nm'])
        offense_desc.append(json_results[x]['ofns_desc'])
        pd_desc.append(json_results[x]['pd_desc'])
        law_cat.append(json_results[x]['law_cat_cd'])
        lat.append(json_results[x]['latitude'])
        lon.append(json_results[x]['longitude'])
    # Update the offset number by 1000
    offset = limit + offset
    # Update the set count
    set += 1
    # Update the records_count variable with count of json_results. Once it hits zero, loop will terminate
    records_count = len(json_results)
    # Delay of 2 second to avoid rate limiting
    time.sleep(2)
# Once loop condition is no longer true, print total number of records    
else: print(f"Total number of records extracted: {records}")

In [ ]:
crime_df = pd.DataFrame({
    "Complaint Number":complaint_number,
    "Reported Date":reported_date,
    "Borough":borough_name,
    "Offense Desc":offense_desc,
    "Police Desc": pd_desc,
    "Law Cat":law_cat,
    "Latitude":lat,
    "Longitude":lon
})

In [ ]:
# DataFrame info
crime_df.info()

In [ ]:
# Save crime_df to CSV file -- @Dalton, change the file name before running
crime_df.to_csv('Output/NPYD_Complaints_01-01-13_12-31-18.csv', index=False)